# Vertical submodule

This helps dealing with data on hybrid coordinates.

Let's search for some model on hybrid coordinates

In [1]:
import cdms2
import genutil
template = '%(root)/%(collection)/%(type)/%(institution)/%(model)/%(experiment)/%(member)/%(table)/%(variable)/%(grid)/%(version)/%(variable)_%(table)_%(model)_%(experiment)_%(member)_%(grid)_%(period).nc'
path = genutil.StringConstructor(template)
path.root = '/global/cscratch1/sd/cmip6'  # this allows to easily change this from one machine to another
path.collection = 'CMIP6'
path.type = 'CMIP'
path.institution = '*'
path.model = '*'
path.experiment = 'historical'
path.member = 'r1i1p1f1'
path.table = 'Amon'
path.variable = 'cl'
path.grid = '*'
path.version = '*'
path.period = '*'

import glob

match = glob.glob(path())
models = set()
for filename in match:
    sp = filename.split("_")
    model = sp[2]
    if not model in models:
        with cdms2.open(filename) as f:
            V = f[path.variable]
            lev = V.getLevel()
            print(len(models), model, lev[:5])
    models.add(model)  # To limit output

0 MRI-ESM2-0 [0.99849961 0.99549962 0.99149897 0.98549794 0.97699573]
1 CESM2-WACCM [-5.96030003e-06 -9.82690018e-06 -1.62018505e-05 -2.67122511e-05
 -4.40410002e-05]
2 CESM2 [ -3.64346569  -7.59481965 -14.35663225 -24.61222    -35.92325002]
3 GISS-E2-1-H [0.94999999 0.93000001 0.91000003 0.88999999 0.87      ]
4 GISS-E2-1-G [0.94999999 0.93000001 0.91000003 0.88999999 0.87      ]
5 MIROC6 [0.99749923 0.99149853 0.98299694 0.97199559 0.95849299]
6 IPSL-CM6A-LR [101202.11  100942.43  100653.414 100331.85   99974.195]
7 CAMS-CSM1-0 [0.99614075 0.98263335 0.95895482 0.92766051 0.89082524]
8 GFDL-CM4 [0.99795    0.99279    0.98540533 0.97563916 0.9632749 ]
9 SAM0-UNICON [0.9925561  0.97632541 0.95748548 0.9361984  0.91264455]
10 BCC-ESM1 [0.9925561  0.97055483 0.92964888 0.86716076 0.78770206]
11 BCC-CSM2-MR [0.99256879 0.9750593  0.94794583 0.90893286 0.86064228]


From this we can see that most models seem to be on some hybrid coordinate system (except IPSL)

Let's use: `MRI-ESM2-0`

In [2]:
filename = match[0]
f = cdms2.open(filename)
cl = f("cl", time=slice(0, 12)) # 1 year
print(cl.shape, cl.getLevel())

(12, 80, 160, 320)    id: lev
   Designated a level axis.
   units:  1
   Length: 80
   First:  0.9984996139408759
   Last:   9.999999776482362e-06
   Other axis attributes:
      axis: Z
      positive: down
      long_name: hybrid sigma pressure coordinate
      standard_name: atmosphere_hybrid_sigma_pressure_coordinate
      formula: p = a*p0 + b*ps
      formula_terms: p0: p0 a: a b: b ps: ps
      realtopology: linear
   Python id:  0x2aaacf932550



The `formula_terms` let us know which variable to recover from the file in order to interpolate to pressure levels

In [3]:
ps = f("ps", time=slice(0,12))
a =f("a")
b = f("b")
p0 = f("p0")
ps.units

'Pa'

First let's reconstruct the pressure

In [4]:
import cdutil
cdutil.reconstructPressureFromHybrid?

Signature: cdutil.reconstructPressureFromHybrid(ps, A, B, Po)
Docstring:
Reconstruct the Pressure field on sigma levels, from the surface pressure


:param Ps: Surface pressure
:param A: Hybrid Conversion Coefficient, such as: p=B.ps+A.Po.
:param B: Hybrid Conversion Coefficient, such as: p=B.ps+A.Po.
:param Po: Hybrid Conversion Coefficient, such as: p=B.ps+A.Po
:param Ps: surface pressure

.. note::

    A and B are 1d sigma levels.
    Po and Ps must have same units.


:returns: Pressure field, such as P=B*Ps+A*Po.

:Example:

    .. doctest:: vertical_reconstructPressureFromHybrid

        >>> P=reconstructPressureFromHybrid(ps,A,B,Po)
File:      /opt/conda/lib/python3.7/site-packages/cdutil-8.1-py3.7.egg/cdutil/vertical.py
Type:      function


In [5]:
p = cdutil.reconstructPressureFromHybrid(ps, a, b, p0)
print(p.shape)

(12, 80, 160, 320)


Now let's interpolate to standrad pressure levels.

In [6]:
cdutil.sigma2Pressure?

Signature:
cdutil.sigma2Pressure(
    A,
    P,
    levels=[100000, 92500, 85000, 70000, 60000, 50000, 40000, 30000, 25000, 20000, 15000, 10000, 7000, 5000, 3000, 2000, 1000],
    status=None,
    axis='z',
)
Docstring:
Log-linear interpolation to convert a field from sigma levels to pressure levels.
Values below surface are masked.

:param A: array on sigma levels
:type A:

:param P: pressure field from TOP (level 0) to BOTTOM (last level)
:type P:

:param levels: pressure levels to interplate to (same units as P), default levels are:
        [100000, 92500, 85000, 70000, 60000, 50000, 40000, 30000, 25000, 20000, 15000, 10000, 7000, 5000,
        3000, 2000, 1000]
:type levels: list

:param axis: axis over which to do the linear interpolation
:type axis: str

.. note::

    P and levels must have same units

:returns: array on pressure levels (levels)

:Example:

    .. doctest:: vertical_logLinearInterpolation

        >>> A=logLinearInterpolation(A,P) # interpolate A using pressure 

The important thing is to make sure the pressure is order from **TOP** to **BOTTOM*

In [7]:
print(p[0,0].max())
print(p[0,-1].min())

103751.25540131927
0.9999999776482361


In our case we will need to *flip* the pressure

In [8]:
cl_levs = cdutil.sigma2Pressure(cl, p[:,::-1], levels=[100000., 92500., 8500.])  # Less levels for faster processing
print(cl_levs.shape)

(12, 3, 160, 320)


In [9]:
print(cl_levs.getLevel())

   id: plev
   Designated a level axis.
   units:  Pa
   Length: 3
   First:  100000.0
   Last:   8500.0
   Other axis attributes:
      axis: Z
      realtopology: linear
   Python id:  0x2aaadb85e7b8



In [10]:
import vcs
x = vcs.init()
x.plot(cl_levs[:,-1])
x.png("cl_interp")

![](cl_interp.png)

Note that `sigma2Pressure` is an alias for `logLinearInterpolation` and performs a logLinear interpolation. A linear interpolation is also availabel with `linearInterpolation`

In [11]:
cl_levs_lin = cdutil.linearInterpolation(cl, p[:,::-1], levels=[100000., 92500., 8500.])
x.clear()
x.plot((cl_levs_lin - cl_levs)[0,-1])
x.png("cl_diff")

![](cl_diff.png)